In [2]:
# 统一导入工具包
import pandas as pd
import csv
import transformers
import torch
from transformers import BertPreTrainedModel, BertModel
from torch import nn
import numpy as np
import os
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

c:\Users\12920\anaconda3\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.4) or chardet (4.0.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn(


In [3]:
# Kagging debug
kagging =True

In [4]:
pd_table = pd.read_csv('./datasets/raw/WikiQA-train.tsv',encoding="utf-8",sep = '\t')

In [5]:
pd_table

,QuestionID,Question,DocumentID,DocumentTitle,SentenceID,Sentence,Label
0,Q1,how are glacier caves formed?,D1,Glacier cave,D1-0,A partly submerged glacier cave on Perito More...,0
1,Q1,how are glacier caves formed?,D1,Glacier cave,D1-1,The ice facade is approximately 60 m high,0
2,Q1,how are glacier caves formed?,D1,Glacier cave,D1-2,Ice formations in the Titlis glacier cave,0
3,Q1,how are glacier caves formed?,D1,Glacier cave,D1-3,A glacier cave is a cave formed within the ice...,1
4,Q1,how are glacier caves formed?,D1,Glacier cave,D1-4,"Glacier caves are often called ice caves , but...",0
...,...,...,...,...,...,...,...
20342,Q3043,what is section eight housing,D2807,Section 8 (housing),D2807-8,A tenant who leaves a subsidized project will ...,0
20343,Q3043,what is section eight housing,D2807,Section 8 (housing),D2807-9,The United States Department of Housing and Ur...,0
20344,Q3044,what is the main type of restaurant,D2808,Category:Types of restaurants,D2808-0,Restaurants categorized by type and informatio...,0
20345,Q3046,what is us dollar worth based on,D2810,History of the United States dollar,D2810-0,U.S. Federal Reserve notes in the mid-1990s,0


In [6]:
def load(filename):
        result = []
        with open(filename, 'r',encoding='utf-8') as csvfile:
            spamreader = csv.reader(csvfile, delimiter='\t', quotechar='"')
            next(spamreader, None)  # skip the headers
            for row in spamreader:
                res = {}
                res['question'] = str(row[1])
                res['answer'] = str(row[5])
                res['label'] = int(row[6])
                result.append(res)
        return result

In [7]:
train_file = load('./datasets/raw/WikiQA-train.tsv')
valid_file = load('./datasets/raw/WikiQA-dev.tsv')
test_file = load('./datasets/raw/WikiQA-test.tsv')

In [8]:
print(len(train_file))
print(len(valid_file))
print(len(test_file))
# Kagging debug
if kagging:
    train_file = train_file[:int(len(train_file)*0.01)]
    valid_file = valid_file[:int(len(valid_file)*0.01)]
    test_file = test_file[:int(len(test_file)*0.01)]
    print(len(train_file))
    print(len(valid_file))
    print(len(test_file))

20347
1130
2349
203
11
23


In [9]:
train_file[:10]

[{'question': 'how are glacier caves formed?',
  'answer': 'A partly submerged glacier cave on Perito Moreno Glacier .',
  'label': 0},
 {'question': 'how are glacier caves formed?',
  'answer': 'The ice facade is approximately 60 m high',
  'label': 0},
 {'question': 'how are glacier caves formed?',
  'answer': 'Ice formations in the Titlis glacier cave',
  'label': 0},
 {'question': 'how are glacier caves formed?',
  'answer': 'A glacier cave is a cave formed within the ice of a glacier .',
  'label': 1},
 {'question': 'how are glacier caves formed?',
  'answer': 'Glacier caves are often called ice caves , but this term is properly used to describe bedrock caves that contain year-round ice.',
  'label': 0},
 {'question': 'How are the directions of the velocity and force vectors related in a circular motion',
  'answer': 'In physics , circular motion is a movement of an object along the circumference of a circle or rotation along a circular path.',
  'label': 0},
 {'question': 'How ar

In [10]:
def padding(sequence, max_length, pad_token=0):
    """
    Pad sequence at the given max_length length.
    pad_token specifies which token should be used to pad.
    pad_position can be 'right' or 'left'.
    """
    padding_length = max_length - len(sequence)
    return sequence + [pad_token] * padding_length

In [11]:
model_path = './datasets/models/bert-pretrain/'
tokenizer = transformers.BertTokenizer.from_pretrained(model_path,do_lower_case=True)
# max_length = 512 
max_length = 64 # Kagging debug
device = torch.device('cpu')
# 如果使用gpu，则注释掉上面一行，使用下面的代码
#device = torch.device('cuda:0')

In [12]:
def tokenize(data, max_length,tokenizer,device):        
    res = []
    for triple in data:
        # 将问题和答案两句话作为输入，使用[SEP]连接，同时转化为one-hot向量
        inputs = tokenizer.encode_plus(
            triple['question'],
            triple['answer'],
            add_special_tokens=True,
            max_length=max_length,
            trunction=True # Kagging debug
        )
        input_ids, token_type_ids = inputs["input_ids"], inputs["token_type_ids"]
        # input_mask.现在还没有padding，因此每个位置都是1
        attention_mask = [1] * len(input_ids)
        # 进行长度补全
        input_ids = padding(
            input_ids,
            max_length,
            pad_token=0
        )
        attention_mask = padding(
            attention_mask,
            max_length,
            pad_token=0
        )
        token_type_ids = padding(
            token_type_ids,
            max_length,
            pad_token=0
        )
        label = triple['label']
        res.append(
             (input_ids, attention_mask, token_type_ids, label)
        )
        all_input_ids = torch.tensor([x[0] for x in res], dtype=torch.int64, device=device)
        all_attention_mask = torch.tensor([x[1] for x in res], dtype=torch.int64, device=device)
        all_token_type_ids = torch.tensor([x[2] for x in res], dtype=torch.int64, device=device)
        all_labels = torch.tensor([x[3] for x in res], dtype=torch.int64, device=device)
        return torch.utils.data.TensorDataset(all_input_ids, all_attention_mask, all_token_type_ids, all_labels)

In [13]:
test = tokenize(test_file,max_length,tokenizer,device)
next(iter(test))

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Keyword arguments {'trunction': True} not recognized.
Keyword arguments {'trunction': True} not recognized.


(tensor([  101,  2129,  3060,  4841,  2020, 17352,  2000,  1996,  2149,   102,
          3060,  7521,  2000,  1996,  2142,  2163,  5218,  2000,  7489,  2000,
          1996,  2142,  2163,  2040,  2024,  2030,  2020, 10342,  1997,  3088,
          1012,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]),
 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor(0))

In [14]:
config = transformers.BertConfig.from_pretrained(model_path)

In [15]:
class BertQA(BertPreTrainedModel):
    def __init__(self, config):
        super(BertQA, self).__init__(config)
        self.num_labels = config.num_labels
        self.bert = BertModel(config)
        # 冻结bert参数,只fine-tuning后面层的参数
        
#         for p in self.parameters():
#             p.requires_grad = False

        self.qa_outputs = nn.Linear(config.hidden_size, 2)
        self.loss_fn = nn.CrossEntropyLoss(reduction='mean')

        self.init_weights()
        
    def forward(self, input_ids=None, attention_mask=None, 
                token_type_ids=None, position_ids=None, 
                head_mask=None, inputs_embeds=None, labels=None):
        # 将数据输入Bert模型，得到Bert提取出的特征
        outputs = self.bert(input_ids,
                            attention_mask=attention_mask,
                            token_type_ids=token_type_ids,
                            position_ids=position_ids,
                            head_mask=head_mask,
                            inputs_embeds=inputs_embeds)
        # 通过全连接网络，将特征转化为一个二维向量，可以看作标签0和1的得分情况
        logits = self.qa_outputs(outputs[0][:, 0, :]).squeeze()
        # 选择得分大的标签作为预测值
        predicted_labels = nn.functional.softmax(logits, dim=-1)
        # 如果输入数据中含有标准答案，就计算loss值（即训练过程）
        if labels is not None:
            loss = self.loss_fn(predicted_labels, labels)
            return loss, predicted_labels
        # 否则返回预测值(测试过程)
        else:
            return predicted_labels
        

In [16]:
model = BertQA.from_pretrained(model_path,config=config)   

Some weights of the model checkpoint at ./datasets/models/bert-pretrain/ were not used when initializing BertQA: ['cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertQA from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertQA from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertQA were not initialized from the model checkpoint at ./datasets/models/bert-pretrain/ and are newly initialized: ['qa_outputs.bia

In [17]:
model.to(device)

BertQA(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
     

In [18]:
batch_size = 8
# Dataset
train_dataset = tokenize(train_file,max_length,tokenizer,device)
# 创建Sampler
train_sampler = torch.utils.data.RandomSampler(train_dataset)
# 通过Dataset和Sampler创建dataloader
train_dataloader = torch.utils.data.DataLoader(train_dataset, sampler=train_sampler, batch_size=batch_size)

Keyword arguments {'trunction': True} not recognized.
Keyword arguments {'trunction': True} not recognized.


In [19]:
epoch_num = 1
learning_rate = 1e-5
adam_epsilon = 1e-8
# 模型路径
# save_path = './datasets/models/'
save_path = './temp/'
if not os.path.exists(save_path):
    os.makedirs(save_path)

In [20]:
optimizer = transformers.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate, eps=adam_epsilon) 

In [23]:
loss_list = []
for epoch in range(epoch_num):
    print("Training epoch {}".format(epoch+1))
    pbar = tqdm(train_dataloader)
    for step, batch in enumerate(pbar):
        # 仅作演示，提前结束训练
        if step == 5:
            torch.save(model.state_dict(),os.path.join(save_path,'best_param.bin'))
            print("Model Saved")
            print("Stopped Early")
            break
        model.train()
        model.zero_grad()
        inputs = {
            'input_ids':      batch[0],
            'attention_mask': batch[1],
            'token_type_ids': batch[2],
            'labels':         batch[3]
        }
        outputs = model(**inputs)
        loss, results = outputs
        loss.backward()
        optimizer.step()
        loss_list.append(loss.item())
        pbar.set_description('[!]Batch loss{:.3f}'.format(loss.item()))

Training epoch 1


  0%|          | 0/1 [00:00<?, ?it/s]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:20                                                                                   │
│                                                                                                  │
│   17 │   │   │   'token_type_ids': batch[2],                                                     │
│   18 │   │   │   'labels':         batch[3]                                                      │
│   19 │   │   }                                                                                   │
│ ❱ 20 │   │   outputs = model(**inputs)                                                           │
│   21 │   │   loss, results = outputs                                                             │
│   22 │   │   loss.backward()                                                                     │
│   23 │   │   optimizer.step()                                                                    │
│                                                                                                  │
│ c:\Users\12920\anaconda3\lib\site-packages\torch\nn\modules\module.py:1194 in _call_impl         │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ in forward:32                                                                                    │
│                                                                                                  │
│   29 │   │   predicted_labels = nn.functional.softmax(logits, dim=-1)                            │
│   30 │   │   # 如果输入数据中含有标准答案，就计算loss值（即训练过程）                            │
│   31 │   │   if labels is not None:                                                              │
│ ❱ 32 │   │   │   loss = self.loss_fn(predicted_labels, labels)                                   │
│   33 │   │   │   return loss, predicted_labels                                                   │
│   34 │   │   # 否则返回预测值(测试过程)                                                          │
│   35 │   │   else:                                                                               │
│                                                                                                  │
│ c:\Users\12920\anaconda3\lib\site-packages\torch\nn\modules\module.py:1194 in _call_impl         │
│                                                                                                  │
│   1191 │   │   # this function, and just call forward.                                           │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1195 │   │   # Do not call functions when jit is used                                          │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                              

In [24]:
print(predicted_labels.size(), labels.size())
loss = self.loss_fn(predicted_labels, labels)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 print(predicted_labels.size(), labels.size())                                                │
│   2 loss = self.loss_fn(predicted_labels, labels)                                                │
│   3                                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'predicted_labels' is not defined